In [2]:
#Dependencies
from splinter import Browser
from splinter.exceptions import ElementDoesNotExist
from bs4 import BeautifulSoup
import time
import pymongo
import requests
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn) 
db = client.goodreads_db
collection = db.items

In [3]:
import json
from ibm_watson import ToneAnalyzerV3
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator

authenticator = IAMAuthenticator('hYXUmMhIZHj11RqZ0B4ltxyIlp5VrpGof7WNbPHiiVH0')
tone_analyzer = ToneAnalyzerV3(
    version='2017-09-21',
    authenticator=authenticator
)

tone_analyzer.set_service_url('https://api.us-south.tone-analyzer.watson.cloud.ibm.com/instances/3a2563d5-be5b-44db-a536-84cb59e18ead')

In [11]:
def star_convert(text):
    if text == 'did not like it':
        return 1
    elif text == 'it was ok':
        return 2
    elif text == 'liked it':
        return 3
    elif text == 'really liked it':
        return 4
    elif text == 'it was amazing':
        return 5
    else:
        return 'N/A'

In [13]:
def tone_convert(tone):
    if tone == 'Anger':
        return 1
    elif tone == 'Fear':
        return 2
    elif tone == 'Joy':
        return 3
    elif tone == 'Sadness':
        return 4
    elif tone == 'Analytical':
        return 5
    elif tone == 'Confident':
        return 6
    elif tone == 'Tentative':
        return 7

In [4]:
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)

In [5]:
url = 'https://www.goodreads.com/genres/most_read/childrens'
browser.visit(url)
html = browser.html
soup = BeautifulSoup(html, 'html.parser')


In [6]:
book_boxes = soup.find_all('div', class_="leftAlignedImage bookBox")
links = []

for box in book_boxes:
    
    link = box.find('a', href=True)['href']
    links.append('https://www.goodreads.com' + link)

In [14]:
for link in links:
    # Retrieve page with the requests module
    response = requests.get(link)
    # Create BeautifulSoup object; parse with 'lxml'
    soup = BeautifulSoup(response.text, 'lxml')
    
    try:
        title = soup.find('h1', id="bookTitle").text

        author = soup.find('a', class_='authorName').text

        results = soup.find_all('div', class_='review')
        
    except:
        title = 'N/A'
        
        author = 'N/A'
        
        results = []
        
    

    for result in results:
        try:
            rating = result.find('span', class_='staticStars notranslate')['title']
            
            rating_num = star_convert(rating)
            
            review = result.find('span',class_="readable").text
            
            tone_analysis = tone_analyzer.tone(
                                                {'text': review},
                                                content_type='text/plain',
                                                sentences = False).get_result()
            
            tone1 = tone_analysis["document_tone"]["tones"][0]["tone_name"]
            
            tone2 = tone_analysis["document_tone"]["tones"][0]["tone_name"]
            
            tone_num = tone_convert(tone1)
            
            
            if (rating and review and tone_analysis and tone1 and tone2):
                
            
                print(title)
                print(author)
                print(rating)
                print(rating_num)
                print(review)
                print(tone1)
                print(tone2)
                print(tone_num)
                
                post = {
                    'title' : title,
                    'author': author,
                    'rating': rating,
                    'rating_num': rating_num,
                    'review': review,
                    'tone1': tone1,
                    'tone2': tone2,
                    'tone_num': tone_num
                }
                
                collection.insert_one(post)

        except Exception as e:
            print("N/A")



      Find Layla

Meg Elison
it was amazing
5

It's sad, heartbreaking but really beautiful.I love you, dear author. The book is so real.Trigger warnings: Bullying, drug addiction, violence, abuseYouTube.Twitter.Viral on the net Neglected kids.Coming of age.Abusive parents.Foster parents.I cannot stop crying.Layla, you need better people in your life as you grow up. And I know you are going to be with better people now. You are such a smart kid. You will grow up beautiful and strong, much stronger than you were. And yassss, if your YouTube c
It's sad, heartbreaking but really beautiful.I love you, dear author. The book is so real.Trigger warnings: Bullying, drug addiction, violence, abuseYouTube.Twitter.Viral on the net Neglected kids.Coming of age.Abusive parents.Foster parents.I cannot stop crying.Layla, you need better people in your life as you grow up. And I know you are going to be with better people now. You are such a smart kid. You will grow up beautiful and strong, much stro


      Find Layla

Meg Elison
it was amazing
5

Truly amazing. It’s 5:25 a.m. and I just finished, and all I can think is it’s been way too long since I cared more about a book than sleep. And this book... wow did it rip my heart out. To Meg Elison, if you read these comments - thank you for telling this story with such grace and compassion. I don’t know if you have first-hand experience with a life like this or if you’re just that amazing of a writer and researcher or all of the above, but whatever the case... you knocked this out of the par
Truly amazing. It’s 5:25 a.m. and I just finished, and all I can think is it’s been way too long since I cared more about a book than sleep. And this book... wow did it rip my heart out. To Meg Elison, if you read these comments - thank you for telling this story with such grace and compassion. I don’t know if you have first-hand experience with a life like this or if you’re just that amazing of a writer and researcher or all of the above, but wha


      Find Layla

Meg Elison
really liked it
4

Great fast reafNot normally a big fan of young adult fiction but this was really good. A story of unimaginable heartache and difficulty with sibling bonds and coping. I want to know more about Layla 

Joy
Joy
3
N/A
N/A

      Find Layla

Meg Elison
really liked it
4

Heart-wrenching. Graphic. Painful. Be careful if you have suffered neglect or abuse because it can be very triggering. A very good read, engaging, but my heart was pounding the whole time.

Sadness
Sadness
4

      Find Layla

Meg Elison
it was amazing
5

Finished this book in one day. Heartbreaking and eye opening.

Sadness
Sadness
4

      Find Layla

Meg Elison
it was amazing
5

Couldn't put it downI don't know if it's because I am a former foster kid who has been through similar things, but I could not put this book down til it was finished. I loved it. I can tell the author has real life experience that informs her writing. The story is realistic, the main character is 


      Find Layla

Meg Elison
it was amazing
5

The secret lives of teenagers.Layla is an unusual 14-year-old. Her grade-point average is a perfect 4.0 and the teachers in the accelerated program she attends marvel at her intelligence and hard work. Clearly, she's a product of a stable, loving home with parents dedicated to giving their daughter every opportunity to reach her full potential. Right?Wrong. Layla lives in an apartment which long ago became a bio-hazard. Ever wonder why a huge percentage of children from low-income families suffe
The secret lives of teenagers.Layla is an unusual 14-year-old. Her grade-point average is a perfect 4.0 and the teachers in the accelerated program she attends marvel at her intelligence and hard work. Clearly, she's a product of a stable, loving home with parents dedicated to giving their daughter every opportunity to reach her full potential. Right?Wrong. Layla lives in an apartment which long ago became a bio-hazard. Ever wonder why a huge perc

ERROR:root:Error in service call
Traceback (most recent call last):
  File "C:\Users\LCDR-Data\anaconda3\lib\site-packages\ibm_cloud_sdk_core\base_service.py", line 208, in send
    response = requests.request(**request, cookies=self.jar, **kwargs)
  File "C:\Users\LCDR-Data\anaconda3\lib\site-packages\requests\api.py", line 60, in request
    return session.request(method=method, url=url, **kwargs)
  File "C:\Users\LCDR-Data\anaconda3\lib\site-packages\requests\sessions.py", line 533, in request
    resp = self.send(prep, **send_kwargs)
  File "C:\Users\LCDR-Data\anaconda3\lib\site-packages\requests\sessions.py", line 646, in send
    r = adapter.send(request, **kwargs)
  File "C:\Users\LCDR-Data\anaconda3\lib\site-packages\requests\adapters.py", line 449, in send
    timeout=timeout
  File "C:\Users\LCDR-Data\anaconda3\lib\site-packages\urllib3\connectionpool.py", line 672, in urlopen
    chunked=chunked,
  File "C:\Users\LCDR-Data\anaconda3\lib\site-packages\urllib3\connectionpool.p

KeyboardInterrupt: 